In [1]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.4.0.post1 requires torch==2.1.2, but you have torch 2.2.2 which is incompatible.
bcembedding 0.1.4 requires transformers<4.37.0,>=4.35.0, but you have transformers 4.40.1 which is incompatible.


In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"

os.environ['http_proxy'] = 'http://127.0.0.1:1080'
os.environ['https_proxy'] = 'http://127.0.0.1:1080'
os.environ['ftp_proxy'] = 'http://127.0.0.1:1080'
os.environ['no_proxy'] = '172.16.x.x,127.0.0.1,localhost'

In [5]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your 

In [2]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied:{info.used // 1024**2} MB.")

### 1. Load Datasets


In [1]:
from datasets import load_dataset

huggingface_data_name = 'neil-code/dialogsum-test'
dataset = load_dataset(huggingface_data_name)
dataset['train'][0]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

{'id': 'train_0',
 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.",
 'summary': "Mr. Smith'

In [2]:
dataset

NameError: name 'dataset' is not defined

### 2. Load model -- quantization

##### BitsAndBytes


In [7]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type = 'nf4', # using BitsAndBytesConfig to load our model in 4-bit format.
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

device_map = {"": 0} # 表示将模型的所有部分（因为键是空字符串，这通常意味着默认或全局设置）放置在设备 6 上。
model_name = 'microsoft/phi-2'
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True,
                                                      )

/home/liangzhu/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.23s/it]


### 3. Tokenization


In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True, 
                                          padding_side='left', 
                                          add_eos_token=True,
                                          add_bos_token=True,
                                          use_fast=False,)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


**model_name:**

- 用途: 指定预训练模型的名称或路径。这可以是本地路径或者是 Hugging Face 模型仓库中的模型名称。
- 示例: "bert-base-uncased", "gpt2", 或者一个本地文件夹路径。

**trust_remote_code (默认为 False):**

- 说明：当你从 Hugging Face 模型库下载一个模型时，该库不仅包含模型的权重和配置文件，还可能包含一些 Python 脚本，这些脚本定义了如何正确地处理、标记化或者预处理数据以适应该模型。如果 trust_remote_code 设置为 True，这些脚本将被加载并执行。
- 用途: 这个参数允许加载远程仓库中的自定义代码。当设置为 True 时，如果远程模型包含自定义的标记化器代码，这些代码将被加载并执行。
  安全考虑: 设置为 True 可能会带来安全风险，因为执行远程代码可能会运行恶意脚本。只在你信任远程源的情况下使用。

**padding_side (默认为 "right"):**

https://ai.stackexchange.com/questions/41485/while-fine-tuning-a-decoder-only-llm-like-llama-on-chat-dataset-what-kind-of-pa

- 用途: 设置标记化后的序列填充方向。可以设置为 "left" 或 "right"。
- 示例: 如果设置为 "left"，填充将会添加到序列的左侧，如在处理右对齐文本时可能需要这样做。

**add_eos_token (一般为模型特定的默认值):**

- 用途: 决定是否在序列的末尾自动添加终止符（EOS，即 End Of Sentence）标记。
- 注意: 并非所有模型都使用 EOS 标记，这依赖于特定模型的预训练过程。

**add_bos_token (一般为模型特定的默认值):**

- 用途: 决定是否在序列的开头自动添加开始符（BOS，即 Beginning Of Sentence）标记。
- 注意: 并非所有模型都使用 BOS 标记，这也依赖于特定模型的预训练过程。

**use_fast (默认为 True):**

- 用途: 决定是否使用快速标记化器。快速标记化器（通常基于 Rust 实现）比纯 Python 标记化器性能更好，但在一些特殊情况下可能需要关闭（例如，当快速标记化器不支持某些特定功能时）。
- 示例: 如果遇到与快速标记化器兼容性的问题，可以设置为 False 使用纯 Python 实现


### 4. zero shot inferencing


In [9]:
def gen(model,p, maxlen=100, sample=True):
    tokens = tokenizer(p, return_tensors="pt")
    # model.generate: 使用模型进行文本生成。
    res = model.generate(**tokens.to("cuda"), # toks.to("cuda"): 将 token 数据移动到 GPU 上进行加速计算。
                         max_new_tokens=maxlen,  # max_new_tokens=maxlen: 生成的最大新 token 数量。
                         do_sample=sample, # do_sample=sample: 是否使用采样策略生成文本。
                         num_return_sequences=1, # num_return_sequences=1: 生成的序列数量。
                         temperature=0.1, # temperature=0.1: 控制生成多样性的温度参数。
                         num_beams=1, # num_beams=1: 束搜索的大小，这里设置为 1 表示不使用束搜索。
                         top_p=0.95, # top_p=0.95: 使用核采样策略，只考虑累计概率至少为 95% 的最高概率 token。
                        ).to('cpu')
  
    return tokenizer.batch_decode(res, skip_special_tokens=True)

In [10]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 10
prompt = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

formatted_prompt = f"Instruct: Summarize the following conversation.\n{prompt}\nOutput:\n"
res = gen(original_model,formatted_prompt,100,)
#print(res[0])
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Summarize the following conversation.
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# attends Brian's birthday pa

### 5. Pre-processing dataset


The dataset cannot be directly employed for fine-tuning. It is essential to format the prompt in a way that the model can comprehend. Referring to the HuggingFace model documentation, it is evident that a prompt needs to be generated using dialogue and summary in the specified format below.

we need to convert the dialog-summary (prompt-response) pairs into explicit instructions for the LLM.


In [ ]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters 
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "Below is an instruction that describles a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "Instruct: Summarize the below conversation."
    RESPONSE_KEY = "### Output:"
    END_KEY = "### END"
    
    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['dialogue']}" if sample["dialogue"] else None
    response = f"{RESPONSE_KEY}\n{sample['summary']}"
    end = f"{END_KEY}"
    
    # 根据给定的模板和数据构建一个文本块列表，每个部分都可能根据其内容是否存在来决定是否包含在最终的列表中。
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]
    
    formatted_prompt = '\n\n'.join(parts)
    sample['text'] = formatted_prompt
    return sample

Now, we will use our model tokenizer to process these prompts into tokenized ones.

Our aim here is to generate input sequences with consistent lengths, which is beneficial for fine-tuning the language model by optimizing efficiency and minimizing computational overhead. It is essential to ensure that these sequences do not surpass the model’s maximum token limit.


In [10]:
from functools import partial

def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ['n_positions', 'max_position_embeddings', 'seq_length']:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            break
    if not max_length:
        max_length = 1024
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch['text'],
        max_length = max_length,
        truncation=True,
    )
    
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """
    # Add prompt to each sample
    dataset = dataset.map(create_prompt_formats)
    
    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['id', 'topic', 'dialogue', 'summary'],
    )
    # 💡 notice: tokenizer 的处理结果如 `input_ids` 等字段会添加到 sample 对象上。
    
    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample['input_ids']) < max_length)
    
    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)
    
    return dataset

In [11]:
max_length = get_max_length(original_model)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['validation'])

2048


In [12]:
print(f"Shapes of the datasets:")
print(f"Training: {train_dataset.shape}")
print(f"Validation: {eval_dataset.shape}")
print(train_dataset)

Shapes of the datasets:
Training: (1999, 3)
Validation: (499, 3)
Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 1999
})


### 6. Preparing the model for QLoRA


Now, let's perform Parameter Efficient Fine-Tuning (PEFT) fine-tuning. PEFT is a form of instruction fine-tuning that is much more efficient than full fine-tuning. PEFT is a generic term that includes Low-Rank Adaptation (LoRA) and prompt tuning (which is NOT THE SAME as prompt engineering!). In most cases, when someone says PEFT, they typically mean LoRA. LoRA, in essence, enables efficient model fine-tuning using fewer computational resources, often achievable with just a single GPU. Following LoRA fine-tuning for a specific task or use case, the outcome is an unchanged original LLM and the emergence of a considerably smaller "LoRA adapter," often representing a single-digit percentage of the original LLM size (in MBs rather than GBs).

During inference, the LoRA adapter must be combined with its original LLM. The advantage lies in the ability of many LoRA adapters to reuse the original LLM, thereby reducing overall memory requirements when handling multiple tasks and use cases.

Note the rank (r) hyper-parameter, which defines the rank/dimension of the adapter to be trained. r is the rank of the low-rank matrix used in the adapters, which thus controls the number of parameters trained. A higher rank will allow for more expressivity, but there is a compute tradeoff.

alpha is the scaling factor for the learned weights. The weight matrix is scaled by alpha/r, and thus a higher value for alpha assigns more weight to the LoRA activations.


In [13]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for  _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 262364160
all model parameters: 1521392640
percentage of trainable model parameters: 17.24%


In [14]:
print(original_model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layern

1. Embedding 层

   - embed_tokens: 词汇表大小为 51200, 每个词被表示为一个 2560 维的向量。
   - embed_dropout： 减少过拟合，通过随机将输入向量的部分元素置零来实现。

2. 编码层 (PhiDecoderLayer) 共有 32 层
   - PhiAttention：rotary_emb - 旋转位置编码
   - PhiMLP：前馈网络（Feed Forward Network, FFN）层（多层感知机）
   - resid_dropout: 残差连接后的 dropout，用于减少过拟合。
3. 输出层 (lm_head)
   - 用于将解码器的输出转换回词汇表大小的维度，以便进行下一个词的预测。这里的输出维度与词汇表大小相同（51200），使得每个输出可以通过 softmax 变换为词汇表中每个词的概率。


In [15]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

config = LoraConfig(
    r = 32,
    lora_alpha = 32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense',
    ],
    bias='none',
    lora_dropout = 0.05, # Conventional
    task_type='CAUSAL_LM',
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

# 2 - Using the prepare_model_for_kbit_training method from PEFT
original_model = prepare_model_for_kbit_training(original_model)
peft_model = get_peft_model(original_model, config)

这两行代码涉及到模型训练过程中的内存优化技术:

1. 梯度检查点（Gradient Checkpointing）

梯度检查点是一种减少训练过程中内存使用的技术，尤其适用于训练大型神经网络。其基本思想是在前向传播过程中不保存所有中间激活值，而只保存选定的几个。在反向传播时，那些未保存的中间激活值会通过重新执行前向传播中的相应部分来重新计算。这种方法可以显著减少存储中间状态所需的内存，但代价是增加了一些计算时间（因为某些操作需要重新计算）。

2. prepare_model_for_kbit_training


In [16]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 20971520
all model parameters: 1542364160
percentage of trainable model parameters: 1.36%


In [17]:
# See how the model looks different now, with the LoRA adapters added:
print(peft_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4

### 7. Train


In [18]:
from transformers import TrainingArguments
import transformers
output_dir = './peft-dialogue-summary-training/final-checkpoint'

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1, # 用于逐渐增加学习率到初始设定值的步骤数。
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    max_steps=500, # 训练过程中的最大步数，不论数据是否已经完全遍历。
    learning_rate=2e-4,
    optim="paged_adamw_8bit", # 优化器：用于减少内存使用并可能加速训练。
    logging_steps=25, # used for tracking the progress of training
    logging_dir="./logs", # 存放日志文件的目录。
    save_strategy="steps", # 模型保存策略。"steps"，表示按步数保存模型。
    save_steps=25, # 指定多少步保存一次模型。
    evaluation_strategy="steps", # 评价策略。这里设置为"steps"，意味着将根据步数来进行模型评估。
    eval_steps=25, # 指定多少步进行一次评估。这里设置为25。与save_steps和logging_steps保持一致可以方便地对比模型的性能。
    do_eval=True, # 是否在训练过程中进行评估。可以及时发现模型的表现问题。这里设置为True，表示将进行评估。
    gradient_checkpointing=True, # 是否启用梯度检查点，以减少内存消耗。这里设置为True。
    report_to="none", # 指定训练过程中的日志和其他输出报告到何处。这里设置为"none"，意味着不报告到任何外部服务或文件。
    overwrite_output_dir = 'True', # 是否覆盖输出目录。在多次训练相同模型或实验时，设置为True可以避免文件冲突。
    group_by_length=True, # 是否根据长度对训练数据进行分组，以优化训练效率。这里设置为True。这有助于减少填充，从而提高数据处理效率。
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
peft_model.to(device)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

max_steps is given, it will override any value given in num_train_epochs


**warmup_steps**: 训练初期逐渐增加到预设初始学习率的步数。适当的 warmup_steps 可以帮助模型在训练初期稳定下来，防止模型在训练开始阶段由于学习率过高而发散。通常，warmup_steps 的设置取决于总训练步数的比例，一般可以设置为总步数的 10%左右。

**per_device_train_batch_size**: 每个设备（如 GPU）在训练过程中的批处理大小。这里设置为 1，表示每个 GPU 每次处理 1 个样本。较小的批次可以减少内存使用，但可能会影响模型性能和训练速度。

**gradient_accumulation_steps**: 每进行 n 次迭代（前向和反向传播）后，才进行一次权重更新。每次迭代处理的样本数为 per_device_train_batch_size。当硬件资源限制了批次大小时，可以通过梯度累积来有效增大有效的批次大小。


In [19]:
peft_training_args.device

device(type='cuda', index=0)

In [20]:
peft_trainer.train()

/home/liangzhu/anaconda3/envs/llm/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/liangzhu/anaconda3/envs/llm/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
25,1.573400,1.372040
50,1.351700,1.347941
75,1.312300,1.346270
100,1.306900,1.335687
125,1.262300,1.341280
150,1.336300,1.331783
175,1.309700,1.328976
200,1.289000,1.326600
225,1.274000,1.327307
250,1.235600,1.326225


/home/liangzhu/anaconda3/envs/llm/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/liangzhu/anaconda3/envs/llm/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/liangzhu/anaconda3/envs/llm/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_

TrainOutput(global_step=500, training_loss=1.2915986137390136, metrics={'train_runtime': 2352.0193, 'train_samples_per_second': 3.401, 'train_steps_per_second': 0.213, 'total_flos': 3.748801409197056e+16, 'train_loss': 1.2915986137390136, 'epoch': 4.0})

In [21]:
torch.cuda.current_device()

0

##### Free memory


In [3]:
print_gpu_utilization()

GPU memory occupied:81619 MB.


In [4]:
del original_model
del peft_trainer

torch.cuda.empty_cache()

NameError: name 'original_model' is not defined

In [5]:
print_gpu_utilization()

GPU memory occupied:81619 MB.


### 8. Evaluate the Model Qualitatively (Human Evaluation)


In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = 'microsoft/phi-2'
base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                  device_map='auto',
                                                  quantization_config=bnb_config,
                                                  trust_remote_code=True,
                                                  use_auth_token=True)

/home/liangzhu/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.98s/it]


In [13]:
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


add_eos_token=True 在训练阶段的使用是为了教会模型识别和生成序列结束的信号，而在使用阶段通常不需要这样做，因为模型已经能够基于其训练来自主生成结束信号。


In [14]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, './peft-dialogue-summary-training/final-checkpoint/checkpoint-1000', torch_dtype=torch.float16, is_trainable=False)

In [15]:
%%time
from transformers import set_seed
set_seed(42)

index = 10
prompt = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

formatted_prompt = f"Instruct: Summarize the following conversation.\n{prompt}\nOutput:\n"
peft_model_res = gen(original_model,formatted_prompt,100,)
peft_model_output = peft_model_res[0].split('Output:\n')[1]
prefix, success, result = peft_model_output.partition('# End')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.

--

### 9. Evaluate the Model Quantitatively (with ROUGE Metric)


ROUGE, or Recall-Oriented Understudy for Gisting Evaluation, is a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing. The metrics compare an automatically produced summary or translation against a reference or a set of references (human-produced) summary or translation.

Let’s now use the ROUGE metric to quantify the validity of summarizations produced by models. It compares summarizations to a “baseline” summary which is usually created by a human. While it’s not a perfect metric, it does indicate the overall increase in summarization effectiveness that we have accomplished by fine-tuning.

To demonstrate the capability of ROUGE Metric Evaluation we will use some sample inputs to evaluate.


In [16]:
# Perform inferences for the sample of the test dataset (only 10 dialogues and summaries to save time).
original_model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map='auto', quantization_config=bnb_config, trust_remote_code=True, use_auth_token=True)

/home/liangzhu/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.11it/s]


In [17]:
import pandas as pd

dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    human_baseline_text_output = human_baseline_summaries[idx]
    prompt = f"Instruct: Summarize the following conversation.\n{dialogue}\nOutput:\n"
    
    original_model_res = gen(original_model,prompt,100,)
    original_model_text_output = original_model_res[0].split('Output:\n')[1]
    
    peft_model_res = gen(ft_model,prompt,100,)
    peft_model_output = peft_model_res[0].split('Output:\n')[1]
    #print(peft_model_output)
    peft_model_text_output, success, result = peft_model_output.partition('#End')
    

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])
df

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

,human_baseline_summaries,original_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,"Person 1: Ms. Dawson, I need you to take a dic...",#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,"Person 1: Ms. Dawson, I need you to take a dic...",#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,"Person 1: Ms. Dawson, I need you to take a dic...",#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,Person2: I'm going to stop driving to work bec...,#Person2# got stuck in traffic again and #Pers...
4,#Person2# decides to follow #Person1#'s sugges...,Person1 and Person2 are discussing the traffic...,#Person2# got stuck in traffic again and #Pers...
5,#Person2# complains to #Person1# about the tra...,Person1 and Person2 are discussing the traffic...,#Person2# got stuck in traffic again and #Pers...
6,#Person1# tells Kate that Masha and Hero get d...,Kate informed that Masha and Hero are getting ...,Masha and Hero are getting divorced. Masha tel...
7,#Person1# tells Kate that Masha and Hero are g...,Kate informed that Masha and Hero are getting ...,Masha and Hero are getting divorced. Masha tel...
8,#Person1# and Kate talk about the divorce betw...,Kate informed that Masha and Hero are getting ...,Masha and Hero are getting divorced. Masha tel...
9,#Person1# and Brian are at the birthday party ...,"Person1 and Person2 are at a party, and Person...",#Person1# brings a birthday gift for Brian and...


In [11]:
!pip install rouge_score

In [18]:
import evaluate

rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2916815228033625, 'rouge2': 0.11149065967039229, 'rougeL': 0.22302258313530804, 'rougeLsum': 0.24365145780467948}
PEFT MODEL:
{'rouge1': 0.34209051890378883, 'rouge2': 0.13197220268881993, 'rougeL': 0.2559244701694561, 'rougeLsum': 0.26872050763009436}


In [19]:
import numpy as np
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 5.04%
rouge2: 2.05%
rougeL: 3.29%
rougeLsum: 2.51%
